In [ ]:
import tkinter as tk

import pynput
import time

import pyautogui

# relative filepath of folder containing 
AUTOMATION_FILEPATH = "Automations/automations.txt"
AUTOMATION_SS_FOLDER = "AutomationScreenshots"



root = tk.Tk()
SCREEN_WIDTH = root.winfo_screenwidth()
SCREEN_HEIGHT = root.winfo_screenheight()

SS_WIDTH = SCREEN_WIDTH // 20
SS_HEIGHT = SCREEN_HEIGHT // 20

# number of seconds to wait while attempting to find an image
WAIT_TIME = 3
CONFIDENCE=0.8


# dimensions for a popup in the middle of the screen
width =  SCREEN_WIDTH // 10
height =  SCREEN_HEIGHT // 8
x_off = 3*SCREEN_WIDTH // 8
y_off = 3*SCREEN_HEIGHT // 8
MIDDLE_POPUP_DIMS = str(width) + "x" + str(height) + "+" + str(x_off) + "+" + str(y_off)


# create the control center, containing buttons where automations are created and run
control_center = tk.Toplevel(root)
control_center.attributes("-topmost", "True")
control_center.configure(bg="purple")
control_center.overrideredirect(True)

# #place control center in bottom left corner
width =  SCREEN_WIDTH // 10
height =  SCREEN_HEIGHT // 8
x_off = width // 2
y_off = 6*height
dimensions = str(width) + "x" + str(height) + "+" + str(x_off) + "+" + str(y_off)
control_center.geometry(dimensions)

# create button for adding an automation
add_automation = tk.Button(control_center, text="Add Automation")
add_automation.grid(row=1, column=0)

## FUNCTIONS TO ADD AUTOMATIONS:

# define as a header the function that loads automations from files
def load_autos():
    pass

# take and save a screenshot on each mouse click
def path_ss(x, y, button, pressed, auto_name, listener):
    if pressed and button==pynput.mouse.Button.left:
        im = pyautogui.screenshot( region=(x-(SS_WIDTH//2), y-(SS_HEIGHT//2), SS_WIDTH, SS_HEIGHT) )

        # get number of screenshot in sequence
        with open(AUTOMATION_FILEPATH, "r") as file:
            content = file.read()
        content_array = content.split()
        last_num = content_array[-1]
        if last_num.isdigit():
            last_num_int = int(last_num)
        else:
            last_num_int = 0
        curr_num_int = last_num_int + 1

        # note that new ss has been taken
        with open(AUTOMATION_FILEPATH, "a") as file:
            file.write(" " + str(curr_num_int))

        im.save(AUTOMATION_SS_FOLDER+"/"+auto_name+str(curr_num_int)+".png")
    if pressed and button==pynput.mouse.Button.right:
        with open(AUTOMATION_FILEPATH, "a") as file:
            file.write(" \n")
        listener.stop()
        load_autos()

# begin collecting path information
def begin_path(event, automation_name_overlay, auto_name_entry):
    auto_name = auto_name_entry.get()
    automation_name_overlay.destroy()
    
    # write name of automation to the file
    with open(AUTOMATION_FILEPATH, "a") as file:
        file.write(auto_name)
    
    listener = pynput.mouse.Listener(on_click= (lambda x, y, button, pressed: path_ss(x, y, button, pressed, auto_name, listener)))
    listener.start()
    
    
    
    
# saving screenshots, remembering paths
def remember_path(event):
    automation_name_overlay = tk.Toplevel(root)
    automation_name_label = tk.Label(automation_name_overlay, text="Name Your Automation")
    auto_name_entry = tk.Entry(automation_name_overlay)
    automation_name_button = tk.Button(automation_name_overlay, text="Begin")
    
#     width =  SCREEN_WIDTH // 10
#     height =  SCREEN_HEIGHT // 8
#     x_off = 3*SCREEN_WIDTH // 8
#     y_off = 3*SCREEN_HEIGHT // 8
#     auto_name_dims = str(width) + "x" + str(height) + "+" + str(x_off) + "+" + str(y_off)
    automation_name_overlay.geometry(MIDDLE_POPUP_DIMS)
    
    automation_name_label.grid(row=2, column=1)
    auto_name_entry.grid(row=3, column=1)
    automation_name_button.grid(row=4, column=1)
    
    automation_name_button.bind("<Button-1>", lambda event: begin_path(event, automation_name_overlay, auto_name_entry))
    
# bind button to the creation of automations
add_automation.bind("<Button-1>", remember_path)


# list of automations, each automation is a tuple with name & number of screenshots
AUTOMATION_LIST = ["No automations"]
AUTOMATION_NUMS = []



def make_autos():
    pass

# load automations:
def load_autos(AUTOMATION_LIST=AUTOMATION_LIST, AUTOMATION_NUMS=AUTOMATION_NUMS):
    while len(AUTOMATION_LIST) > 0:
        AUTOMATION_LIST.pop()
    with open(AUTOMATION_FILEPATH, "r") as file:
        line = file.readline()
        while line:
            words = line.split()
            name = words[0]
            num = words[-1]
            if num.isdigit():
                num_int = int(words[-1])
                AUTOMATION_LIST.append(name)
                AUTOMATION_NUMS.append(num_int)
            line = file.readline()
    if len(AUTOMATION_LIST) == 0:
        AUTOMATION_LIST.append("No automations")
#     print(AUTOMATION_LIST)
    
    make_autos()
    return



global autos_menu
global autos_run
global selected_auto
def make_autos():
    global autos_menu
    global autos_run
    global selected_auto
    try:
#         selected_auto.destroy()
        autos_menu.destroy()
        autos_run.destroy()
    except Exception as e:
#         print(e)
        pass
    selected_auto = tk.StringVar(root)
    selected_auto.set(AUTOMATION_LIST[0])  # Set the default option

    autos_menu = tk.OptionMenu(control_center, selected_auto, *AUTOMATION_LIST)
    autos_menu.grid(row=2, column=0)

    autos_run = tk.Button(control_center, text="Run")
    autos_run.grid(row=2, column=1)
    autos_run.bind("<Button-1>", lambda event: run_auto(event, selected_auto))

# load automations on boot
load_autos()
    
# running automations

def run_auto(event, selected_auto):
    auto_name = selected_auto.get()
    
    # get number of screenshots associated with automation
    i = AUTOMATION_LIST.index(auto_name)
    num_ss = AUTOMATION_NUMS[i]
    
    # click on each screenshot sequentially
    for i in range(1, num_ss+1):
        fail = 0
        success = False
        while fail < WAIT_TIME:
            try:
                im_path = AUTOMATION_SS_FOLDER + "/" + auto_name + str(i) + ".png"
                x, y = pyautogui.locateCenterOnScreen(im_path, confidence=CONFIDENCE)
                pyautogui.moveTo(x, y)
                pyautogui.click()
                fail = WAIT_TIME
                success = True
            except Exception as e:
#                 print(e)
                time.sleep(1)
                fail += 1
        if not success:
            
            failure_notification = tk.Toplevel(root)
            failure_label = tk.Label(failure_notification, text="Automation could not finish")
            failure_notification.attributes("-topmost", True)
            failure_notification.geometry(MIDDLE_POPUP_DIMS)
            failure_label.pack()
            return
    
    success_notification = tk.Toplevel(root)
    success_label = tk.Label(success_notification, text="Automation successful")
    success_notification.attributes("-topmost", True)
    success_notification.geometry(MIDDLE_POPUP_DIMS)
    success_label.pack()
    
    success_notification.after(2000, success_notification.destroy)
            
    
root.mainloop()

automation succeeded
automation succeeded
